In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-06 00:44:19.424118


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-06T00:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-03-06 00:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-02-05 00:00:00+0000', tz='UTC')

In [9]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    #rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]
    rides_from["pickup_datetime"] = rides_from["pickup_datetime"].dt.tz_localize(None)
    historical_from_date = historical_from_date.replace(tzinfo=None)


    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        #rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides_to["pickup_datetime"] = rides_to["pickup_datetime"].dt.tz_localize(None)
        historical_to_date = historical_to_date.replace(tzinfo=None)
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [10]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [11]:
rides

,pickup_datetime,pickup_location_id
1284755,2025-02-12 16:25:44,2
1591456,2025-02-15 16:56:40,2
3845842,2025-03-08 18:11:37,2
3878691,2025-03-08 23:16:48,2
4665647,2025-03-16 17:46:48,2
...,...,...
6046121,2025-03-30 23:47:33,263
6046621,2025-03-30 23:48:42,263
6046910,2025-03-30 23:55:09,263
6046426,2025-03-30 23:57:35,263


In [12]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [13]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-30 00:00:00,2,0
1,2025-01-30 01:00:00,2,0
2,2025-01-30 02:00:00,2,0
3,2025-01-30 03:00:00,2,0
4,2025-01-30 04:00:00,2,0
...,...,...,...
368635,2025-03-30 19:00:00,263,98
368636,2025-03-30 20:00:00,263,88
368637,2025-03-30 21:00:00,263,70
368638,2025-03-30 22:00:00,263,28


In [14]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368640 entries, 0 to 368639
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         368640 non-null  datetime64[ns]
 1   pickup_location_id  368640 non-null  int16         
 2   rides               368640 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 4.2 MB


In [15]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-05 19:45:00,176 INFO: Initializing external client
2025-03-05 19:45:00,176 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 19:45:01,348 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214686


In [16]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████| Rows 368640/368640 | Elapsed Time: 00:34 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/sp25_taxi_sahilsvachhani/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741125083997) to trigger the materialization job again.


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)